In [2]:
%matplotlib inline
# packages to load 
# Check the versions of libraries
# Python version
import warnings
warnings.filterwarnings('ignore')
import sys
print('Python: {}'.format(sys.version))

import scipy
print('scipy: {}'.format(scipy.__version__))

import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


import random as rn
import math

from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from itertools import product

TRAIN_FILE_PATH = "data/X_train.csv"
TARGET_FILE_PATH =  "data/y_train.csv"
TEST_FILE_PATH = "data/X_test.csv"

seed=42
np.random.seed(seed)
rn.seed(seed)


Python: 3.6.6 |Anaconda, Inc.| (default, Jun 28 2018, 17:14:51) 
[GCC 7.2.0]
scipy: 1.1.0


# Used functions

In [4]:
def make_submission(filename, predictions):
    test_data =  pd.read_csv(TEST_FILE_PATH)
    test_data["y"] = predictions
    test_data[["id", "y"]].to_csv("submissions/"+filename, index= False)

# Load train and test set

In [3]:
train_data = pd.read_csv(TRAIN_FILE_PATH)
train_data.drop("id", axis=1, inplace=True)

Y_train = pd.read_csv(TARGET_FILE_PATH)
Y_train.drop("id", axis=1, inplace = True)

test_data =  pd.read_csv(TEST_FILE_PATH)
id_test = test_data.columns[0]
test_data.drop("id", axis=1, inplace=True)

# Create model and do submission

In [6]:
X_train = train_data.values
y_train = Y_train.values.ravel()
X_test = test_data.values

n_samples = y_train.shape[0]
n_features = X_train.shape[1]

class_weight = n_samples / (3 * np.bincount((y_train).astype(int)))
# svc defaults: Cross Validation: BMAC: 0.6992 (+/- 0.0314)
class_weights0 = { 
    0 : class_weight[0],
    1 : class_weight[1],
    2 : class_weight[2]
}

# tuned with bayesian optimization: Cross Validation: BMAC 0.7004 (+/- 0.0305)
class_weights1 = { 
    0: 3.2734,
    1: 0.6869,
    2: 2.6268
}

# tuned with bayesian optimization: Cross Validation:  BMAC 0.7006 (+/- 0.0248)
class_weights2 = { 
    0 : 2.4720,
    1 : 0.4549,
    2 : 3.8704
}  

clf0 = svm.SVC(class_weight=class_weights0)
clf1 = svm.SVC(class_weight=class_weights1)
clf2 = svm.SVC(class_weight=class_weights2)
eclf = VotingClassifier(estimators=[('clf0', clf0), ('clf1', clf1), ('clf2', clf2)], voting='hard')

eclf.fit(X_train, y_train)
pred = eclf.predict(X_test)

#  BMAC_ENSEMBLE score: 0.7018 std: 0.0328
make_submission('final.csv', pred)